In [1]:
from pyspark import SparkFiles

In [2]:
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

camera_df =  spark.read.option('header', 'true').csv("file://"+SparkFiles.get("amazon_reviews_us_Camera_v1_00.tsv.gz"), inferSchema=True, sep='\t')
camera_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
marketplace|customer_id| review_id|product_id|product_parent| product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase| review_headline| review_body| review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
 US| 2975964|R1NBG94582SJE2|B00I01JQJM| 860486164|GoPro Rechargeabl...| Camera| 5| 0| 0| N| Y| Five Stars| ok|2015-08-31 00:00:00|
 US| 23526356|R273DCA6Y0H9V7|B00TCO0ZAA| 292641483|Professional 58mm...| Camera| 5| 0| 0| N| Y| Love it!!!|Perfect, even stu...|2015-08-31 00:00:00|
 US| 52764145| RQVOXO7WUOFK6|B00B7733E0| 75825744|Spy Tec Z12 Motio...| Camera| 2| 1| 1| N| Y|Another Motion De...|If the words, &#3...|2015-08-31 00:00:00|
 US| 47348933|R1KWKSF21PO6HO|B006ZN4U34| 789352955|Celestron UpClose...| Camera| 5| 0| 0| N| Y|Exactly what I wa...|Exactly what I wa...|2015-08-31 00:00:00|
 US| 33680700|R38H3UO1J190GI|B00HUEBGMU| 19067902|Vidpro XM-L Wired...| Camera| 5| 1| 1| N| Y|Good mic at a Goo...|I will look past ...|2015-08-31 00:00:00|
 US| 30301059|R3NPIFKLR19NQA|B008MW6Y12| 597683407|NIX 8 inch Hi-Res...| Camera| 3| 0| 0| N| Y|The controls are ...|The controls are ...|2015-08-31 00:00:00|
 US| 28282645|R3MBE6UCH3435E|B00TE8XKIS| 35563334|Polaroid ZIP Mobi...| Camera| 3| 8| 8| N| N|Fun toy for makin...|The printer came ...|2015-08-31 00:00:00|
 US| 502818|R2E7A4FF0PVY5Q|B00ZKDUFBQ| 555190742|GeekPro 2.0-Inch ...| Camera| 5| 0| 1| N| Y| Five Stars|Great camera for ...|2015-08-31 00:00:00|
 US| 1481233| R3R8JDQ2BF4NM|B010BZ7S2Q| 129544315|Sony HDR-AZ1VR Ac...| Camera| 5| 0| 2| N| Y| Five Stars|Product is very g...|2015-08-31 00:00:00|
 US| 27885926|R1YND4BS823GN5|B00HRXSSRA| 708418657|ChiliPower DMW-BL...| Camera| 1| 0| 0| N| Y| Sucky.|Lasted a few hour...|2015-08-31 00:00:00|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
only showing top 10 rows

In [3]:
# Count 
camera_df.count()

Out[6]: 1801974

In [4]:
from pyspark.sql.functions import to_date

# review_id DataFrame
review_id_df = camera_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show(10)

+--------------+-----------+----------+--------------+-----------+
 review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
R1NBG94582SJE2| 2975964|B00I01JQJM| 860486164| 2015-08-31|
R273DCA6Y0H9V7| 23526356|B00TCO0ZAA| 292641483| 2015-08-31|
 RQVOXO7WUOFK6| 52764145|B00B7733E0| 75825744| 2015-08-31|
R1KWKSF21PO6HO| 47348933|B006ZN4U34| 789352955| 2015-08-31|
R38H3UO1J190GI| 33680700|B00HUEBGMU| 19067902| 2015-08-31|
R3NPIFKLR19NQA| 30301059|B008MW6Y12| 597683407| 2015-08-31|
R3MBE6UCH3435E| 28282645|B00TE8XKIS| 35563334| 2015-08-31|
R2E7A4FF0PVY5Q| 502818|B00ZKDUFBQ| 555190742| 2015-08-31|
 R3R8JDQ2BF4NM| 1481233|B010BZ7S2Q| 129544315| 2015-08-31|
R1YND4BS823GN5| 27885926|B00HRXSSRA| 708418657| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows

In [5]:
# Products DataFrame
products_df = camera_df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show(10)

+----------+--------------------+
product_id| product_title|
+----------+--------------------+
B00L9ZF5BY|Flylinktech Actio...|
B00017LSF8|B+W Clear UV Haze...|
B00CGTN2N0|Queen Spy Mini U8...|
B002POMZWU|Usb cable for kod...|
B00FEY2BCW|Camera Case Eveca...|
B0131XNIDS|*5-pack* microSue...|
B004OUMTT2|Celestron PowerSe...|
B004HY3XOA|58MM UV Lens Filt...|
B00LBFKLRU|(Create a generic...|
B007QXZ6D0|Vello Cinch Strap...|
+----------+--------------------+
only showing top 10 rows

In [6]:
# Customers DataFrame
customers_df = camera_df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show(10)

+-----------+--------------+
customer_id|customer_count|
+-----------+--------------+
 52695798| 1|
 48363612| 5|
 46909180| 1|
 45595220| 1|
 50372387| 2|
 9731896| 1|
 24540309| 1|
 2019000| 1|
 50798385| 5|
 37669883| 1|
+-----------+--------------+
only showing top 10 rows

In [7]:
# Vine DataFrame
vine_df = camera_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+
 review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
R1NBG94582SJE2| 5| 0| 0| N|
R273DCA6Y0H9V7| 5| 0| 0| N|
 RQVOXO7WUOFK6| 2| 1| 1| N|
R1KWKSF21PO6HO| 5| 0| 0| N|
R38H3UO1J190GI| 5| 1| 1| N|
R3NPIFKLR19NQA| 3| 0| 0| N|
R3MBE6UCH3435E| 3| 8| 8| N|
R2E7A4FF0PVY5Q| 5| 0| 1| N|
 R3R8JDQ2BF4NM| 5| 0| 2| N|
R1YND4BS823GN5| 1| 0| 0| N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows

In [8]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-1.cwoll1t89gv9.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"root", 
          "password": "200233032p", 
          "driver":"org.postgresql.Driver"}

In [9]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [10]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [11]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [12]:
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)